In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

import random
import math

In [2]:
df = pd.read_csv('housing.csv', header=None, sep='\s+')  # sep='\s+' or  sep='\t' or
column_names = [
    'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
    'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV'
]


df.columns = column_names
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\akaku\AppData\Local\Temp\ipykernel_15808\743828371.py:1: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('housing.csv', header=None, sep='\s+')  # sep='\s+' or  sep='\t' or


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
X = df.drop(df.columns[-1], axis=1).to_numpy()
y = df['MEDV'].to_numpy()

In [4]:
test_size = 0.25

row_count = X.shape[0] 
row_count_in_samples = int(row_count * test_size)

indixes = np.random.permutation(row_count)

test_indices = indixes[:row_count_in_samples]
train_indices = indixes[row_count_in_samples:]


X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

Linear Regression 

In [ ]:
def linear_regression(
        X: np.ndarray,
        y: np.ndarray,
        learning_rate: float = 0.01,
        iterations: int = 5000,
        coef_umensheniya_learning_rate: float = 0.001,
        epsilon: float = 1e-4
    ) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Линейная регрессия.

    :param X: Исходная матрица.
    :param y: Целевые значения.
    :param learning_rate: Начальная скорость.
    :param iterations: максимальное количество итераций.
    :param coef_umensheniya_learning_rate: Коэффициент уменьшения скорости сходимости.
    :param epsilon: показатель качества модели для досрочного выхода из обучения.
    :return: Кортеж: 
        - weights: Веса.
        - X_mean: Средние значения.
        - X_std: Стандартные отклонения.
    """
    # нормализация признаков
    X_mean = np.mean(X, axis=0 )
    X_std = np.std(X, axis=0)
    # Избегаем деления на ноль
    X_std[X_std == 0] = 1
    X_normalized = (X - X_mean) / X_std
    
    # Добавление столбца единиц для свободного члена
    row, col = X_normalized.shape
    X_b = np.column_stack([np.ones(row), X_normalized])

    weights = np.random.randn(col + 1)

    for i in range(iterations):
        y_pred = X_b @ weights
        error = y_pred - y

        gradient = (2 / row) * X_b.T @ error
        current_lr = learning_rate * (1 / (1 + coef_umensheniya_learning_rate * i))
        old_weights = weights.copy()
        weights = weights - current_lr * gradient

        # Проверяем изменение весов
        weight_change = np.linalg.norm(weights - old_weights)
        
        if weight_change < epsilon:
            print(f"Сходимость достигнута на итерации {i}, изменение весов: {weight_change:.2e}")
            break

        if i == iterations - 1:
            print(f"Заданная точность НЕ достигнута за {iterations} шагов, последнее изменение весов: {weight_change:.2e}")
    
    return weights, X_mean, X_std

def predict(X, weights, X_mean, X_std):
    """
    Предсказание.
    
    :param X: Исходная матрица.
    :param weights: Веса.
    :param X_mean:Средние значения.
    :param X_std: Стандартные отклонения.
    :return: Предсказание.
    """
    X_normalized = (X - X_mean) / X_std
    row = X_normalized.shape[0]
    X_b = np.column_stack([np.ones(row), X_normalized])
    return X_b @ weights

In [6]:
weights, X_mean, X_std = linear_regression(X_train, y_train)


y_predict = predict(X_test, weights, X_mean, X_std)

Сходимость достигнута на итерации 3081, изменение весов: 1.00e-04


sklearn

In [7]:
regression = LinearRegression()
regression.fit(X_train, y_train)
y_predict_sk = regression.predict(X_test)

Результаты

In [8]:
for i in range(len(y_predict)):
    print(f'{y_predict[i]}\t{y_predict_sk[i]}\t{y_test[i]}')

43.78732542044757	43.77970440936291	50.0
27.14101426489541	27.158567713877563	24.8
35.01292337298355	34.974994008933486	34.9
27.93899831199134	27.93785157137144	23.9
20.139454477193187	20.152725409770095	20.5
12.765617345232714	12.77038434540193	12.8
13.129630151688104	13.140267407080653	19.0
25.04424835381536	25.050865005070186	25.0
20.314255798055036	20.322233432374027	17.7
30.465635803667425	30.48198686211987	29.1
22.28912834831496	22.20483035720266	16.5
15.693908225239472	15.689023775020988	18.4
31.148800801359823	31.15117380327668	34.7
35.129877585231775	35.129725849029334	50.0
3.0173962318125636	3.008035450154992	14.4
31.28774386055054	31.276018277735055	24.8
20.304688444449894	20.391131328160608	18.8
17.898461683382852	17.906235436985853	14.2
27.487827340168817	27.49049424738297	22.8
33.070364678825094	33.07578344991995	37.2
21.336273063052577	21.31832865984696	17.0
31.238873955450465	31.20184722508327	37.0
20.294222636051785	20.270540865779225	21.8
34.870450132384654	34.8751077